## 지진 발생 지역 클러스터링

Scikit-learn과 2018~2024 지진 데이터 세트를 사용하여 클러스터를 만드는 방법에 대해 알아봅니다.

K-Means clustering은 일련의 관측치를 사용하여 데이터 그룹을 'k'개의 클러스터로 나누고 분할하는 데 사용됩니다. 각 관측치는 가장 가까운 '평균' 또는 클러스터의 중심점에 가장 가까운 주어진 데이터 포인트를 그룹화하는 데 사용됩니다.


In [ ]:
%pip install seaborn

필요한 라이브러리를 설치하고 데이터를 불러옵니다.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


df = pd.read_csv("./EQK_지진정보_20240420233456.csv", encoding='EUC-KR')
df.head()

UnicodeDecodeError: 'euc_kr' codec can't decode byte 0x93 in position 21101: illegal multibyte sequence

위도와 경도 데이터만을 사용할 것입니다

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터셋 로드
df = pd.read_csv("./EQK_지진정보_20240420233456.csv", encoding='EUC-KR')

# 진앙이 0 초과인 데이터 필터링
df = df[df['진앙(km)'] > 0]

# 위도와 경도에 대한 막대 그래프
plt.figure(figsize=(14, 5))

# 위도 분포 막대 그래프
plt.subplot(1, 2, 1)
plt.hist(df[(df['위도'] >= 33) & (df['위도'] <= 43)]['위도'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Latitude', fontsize=15)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 경도 분포 막대 그래프
plt.subplot(1, 2, 2)
plt.hist(df[(df['경도'] >= 124) & (df['경도'] <= 132)]['경도'], bins=20, color='lightgreen', edgecolor='black')
plt.title('Distribution of Longitude', fontsize=15)
plt.xlabel('Longitude', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


In [ ]:
df.head()

box plots을 활용하여 이상치를 확인합니다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터셋 로드
df = pd.read_csv("./EQK_지진정보_20240420233456.csv", encoding='EUC-KR')

# 진앙이 0 초과인 데이터 필터링
df = df[df['진앙(km)'] > 0]

# 열 이름을 영어로 변경
df = df.rename(columns={'규모': 'Magnitude', '진앙(km)': 'Epicenter (km)', '위도': 'Latitude', '경도': 'Longitude'})

plt.figure(figsize=(20, 20), dpi=200)

plt.subplot(4, 3, 1)
sns.boxplot(x='Magnitude', data=df)

plt.subplot(4, 3, 2)
sns.boxplot(x='Epicenter (km)', data=df)

plt.subplot(4, 3, 3)
sns.boxplot(x='Latitude', data=df)

plt.subplot(4, 3, 4)
sns.boxplot(x='Longitude', data=df)

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# LabelEncoder 객체 생성
le = LabelEncoder()

# 데이터셋에서 클러스터링에 사용할 열 선택
X = df[['Latitude', 'Longitude']]

# 결측치 제거
X = X.dropna()

# 데이터 표준화 및 Label Encoding
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.cluster import KMeans

# 클러스터 개수
n_clusters = 3 

# 시드 설정
random_state = 0

# KMeans 모델 생성 및 훈련
kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
kmeans.fit(X_scaled)

# 각 데이터 포인트에 대한 클러스터 예측
y_cluster_kmeans = kmeans.predict(X_scaled)
y_cluster_kmeans

In [ ]:
from sklearn import metrics
score = metrics.silhouette_score(X, y_cluster_kmeans)
score

실루엣 점수는 클러스터링의 성능을 측정하는 지표 중 하나입니다. 실루엣 점수는 각 데이터 포인트가 같은 클러스터 내의 다른 데이터포인트와 얼마나 가깝고(f) 다른 클러스터의 데이터포인트와 얼마나 먼지를 나타내는 값입니다. 실루엣 점수는 -1부터 1까지의 값을 가지며, 1에 가까울수록 클러스터링의 성능이 좋다고 평가됩니다.

주어진 실루엣 점수는 0.454입니다. 1에 가까울수록 좋지만, 0.5 이상의 점수는 일반적으로 좋은 클러스터링이라고 평가됩니다.
따라서 주어진 클러스터링 결과는 상당히 좋은 편이며, 데이터셋이 잘 클러스터링 되었다고 평가할 수 있습니다.

In [ ]:
from sklearn.cluster import KMeans

# 클러스터 개수별 WCSS 계산
wcss = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(X_scaled)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(x=range(1, 11), y=wcss, marker='o', color='red')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# KMeans 모델 생성 및 학습
kmeans = KMeans(n_clusters=3)
kmeans.fit(X_scaled)

# 클러스터링 결과 라벨
labels = kmeans.labels_

# 클러스터링 결과 시각화
plt.figure(figsize=(10, 7))
plt.scatter(X['Latitude'], X['Longitude'], c=labels, cmap='viridis')  # 클러스터링 결과를 산점도로 시각화
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red', marker='x')  # 각 클러스터의 중심점을 표시
plt.title('Earthquake Clusters')
plt.xlabel('Latitude')
plt.ylabel('Longitude')

# 위도와 경도의 범위 설정
plt.xlim(33, 43)
plt.ylim(124, 132)

plt.show()
